In [1]:
import os
import random
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
# Neural Network Class

# Fully connected neural network with number of layers, neurons, input size, output size and activation function as parameters

class NeuralNetwork(nn.Module):
    def __init__(self, num_layers, num_neurons, input_size, output_size, activation_function):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.activation_function = activation_function

        # Input layer
        self.layers.append(nn.Linear(input_size, num_neurons))

        # Hidden layers
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(num_neurons, num_neurons))

        # Output layer
        self.layers.append(nn.Linear(num_neurons, output_size))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation_function(layer(x))
        x = self.layers[-1](x)
        return x

## define the activation functions: tanh() and sin()
def tanh(x):
    return torch.tanh(x)

def sin(x):
    return torch.sin(x)


In [ ]:
# initial condition 
def ic_func(x):
    return np.cos(x[:, 0:1]) + 0.1 * np.sin(2 * x[:, 0:1])


In [ ]:
# boundary condition

## periodic boundary condition


In [ ]:
# Loss Function construction

## Residual loss function
def Residual_NN(x, u, t, u_t, u_xx):
    # Compute the residual of the PDE
    f = u_t + u_xx - np.sin(t) * np.cos(x)
    return f

## Boundary loss function
def Boundary_NN(x, t, u):
    # Compute the boundary condition loss
    u_bc = torch.zeros_like(u)
    return u - u_bc

## Initial loss function
def Initial_NN(x, u):
    # Compute the initial condition loss
    u_ic = ic_func(x)
    return u - u_ic

## training loss function
def train_loss(data, model):
    x = x.requires_grad_()
    t = t.requires_grad_()
    u = u.requires_grad_()
    u_t = u_t.requires_grad_()
    u_xx = u_xx.requires_grad_()


    # Compute the total loss
    res_loss = Residual_NN(x, u, t, u_t, u_xx)
    bc_loss = Boundary_NN(x, t, u)
    ic_loss = Initial_NN(x, u)
    total_loss = res_loss + bc_loss + ic_loss
    return total_loss, res_loss,  ic_loss


In [ ]:
# training process
epochs = 5000

loss_history = []
ode_loss_history = []
initial_loss_history = []

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = NeuralNetwork(num_layers=4, num_neurons=50, input_size=1, output_size=1, activation_function=tanh)
print(model)

x_train = torch.linspace(-2, 2, 100).view(-1, 1)  # Training points

for epoch in range(epochs):
    optimizer.zero_grad()
    total_loss, ode_loss, initial_loss = train_loss(model, x_train)
    total_loss.backward()
    optimizer.step()

    
    loss_history.append(total_loss.item())
    ode_loss_history.append(ode_loss.item())
    initial_loss_history.append(initial_loss.item())

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item():.6f}")


